In [ ]:
import opt_note.scsp as scsp
from ortools.sat.python import cp_model
import automaton_cpsat_test

In [ ]:
import marimo as mo
import nbformat

# ALPHABET から削り出すやつを整数線形計画でやる

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        chars = "".join(sorted(list(set("".join(instance)))))
        max_len = len(chars) * max(len(s) for s in instance)

        cpmodel = cp_model.CpModel()

        valids = [cpmodel.new_bool_var("") for _ in range(max_len)]
        cvars = [
            [
                cpmodel.new_int_var(0, max(len(s) for s in instance) - 1, "")
                for c in s
            ]
            for s in instance
        ]

        for sidx, s in enumerate(instance):
            for cidx, c in enumerate(s):
                if cidx == 0:
                    continue
                cpmodel.add(
                    len(chars) * cvars[sidx][cidx - 1] + chars.index(s[cidx - 1])
                    < len(chars) * cvars[sidx][cidx] + chars.index(s[cidx])
                )

        for sidx, s in enumerate(instance):
            for cidx, c in enumerate(s):
                cpmodel.add_element(
                    len(chars) * cvars[sidx][cidx] + chars.index(c),
                    valids,
                    1,
                )

        cpmodel.minimize(sum(valids))

        self.instance = instance
        self.chars = chars
        self.cpmodel = cpmodel
        self.cpsolver = cp_model.CpSolver()
        self.valids = valids
        self.status: cp_model.cp_model_pb2.CpSolverStatus | None = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.cpsolver.parameters.log_search_progress = log
        if time_limit is not None:
            self.cpsolver.parameters.max_time_in_seconds = time_limit
        self.status = self.cpsolver.solve(self.cpmodel)

        return self

    def to_solution(self) -> str | None:
        if self.status not in {
            cp_model.cp_model_pb2.OPTIMAL,
            cp_model.cp_model_pb2.FEASIBLE,
        }:
            return None

        solution = ""
        for idx, valid in enumerate(self.valids):
            if self.cpsolver.boolean_value(valid):
                solution += self.chars[idx % len(self.chars)]

        return solution

In [ ]:
def bench1(instance: list[str]) -> None:
    model = automaton_cpsat_test.Model(instance, False).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution status: {model.cpsolver.status_name()}")
    print(f"bset bound: {model.cpsolver.best_objective_bound}")

In [ ]:
def bench2(instance: list[str]) -> None:
    model = Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution status: {model.cpsolver.status_name()}")
    print(f"bset bound: {model.cpsolver.best_objective_bound}")

In [ ]:
instance01 = scsp.example.load("uniform_q26n004k015-025.txt")

In [ ]:
bench1(instance01)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 64) ---
 Sol: iougjteiqvafkozglnbkruxdhmpxcdinvybchostuoqvgoxzprvxinsnsbgpxflp
str1: -----t------k--g-n-k-u--hmpx---n----h--t--q-g-xz--vxi-s---------
str2: io--j--iq--f-o--lnb---x----xc---v-----s-u-q-----p-v-i-s-sb--xf--
str3: --u-------------l-----------c-in-y-c-os--o-v-o-zp----------p--lp
str4: i--g--e--va---zg--b-r--d-----d----bc--s----v-----rv--n-n--g--f--

solution is feasible: True
solution status: FEASIBLE
bset bound: 52.0


In [ ]:
bench2(instance01)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: tuklcignyckosuhjmoeiqvafozppglnbrxddxbcsvrsvnhtunqgpxzvxissbxf
str1: t-k---gn--k--uh-m---------p------x----------nht--qg-xzvxis----
str2: -----i-----o---j---iq--fo----lnb-x--x-c-v-s----u-q-p--v-issbxf
str3: -u-lci-nyc-os----o---v--ozpp-l---------------------p----------
str4: -----ig-----------e--va--z--g--br-dd-bcsvr-vn---n-g----------f

solution is feasible: True
solution status: OPTIMAL
bset bound: 62.0


In [ ]:
instance02 = scsp.example.load("uniform_q26n008k015-025.txt")

In [ ]:
bench1(instance02)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 111) ---
 Sol: iruxlowjptbdeikvxydgnqbcfikoruvdhmprzefjlnrtvabxyzgnxbcertvdosdhtubcmopsvzbmqrvgnotxzdfpuvxinqsvwcdgpsblpxbfheo
str1: ---------t----k----gn-----k--u--hmp------------x---n-----------ht-----------q--g---xz----vxi--s----------------
str2: i----o-j-----i-------q--f--o------------ln----bx----x-c---v--s---u----------q----------p-v-i--s------sb--x-f---
str3: --u-l------------------c-i---------------n------y-----c-----os-------o--v--------o--z--p------------p--lp------
str4: i------------------g-----------------e------va---zg--b--r--d--d---bc---sv----rv-n-----------n------g-------f---
str5: --------p--------y----------------p-----l-r------z--x------------u-c--p----mq-vg--t--df-u--i---v-cd--sb--

In [ ]:
bench2(instance02)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 103) ---
 Sol: iorxgjpwxbdeiqtvyafkopzglnrbdkruxlxchmzpvxinsuycfhjptmoqvdgptxdefrzuvxisvbcdmsbpvroqvnozinpgpxblwbdefhp
str1: --------------t----k---g-n---k-u----hm-p-x-n-----h--t--q--g--x----z-vxis-------------------------------
str2: io---j------iq----f-o---ln-b----x-xc----v---su---------q---p--------v-is-----sb--------------x------f--
str3: -------------------------------u-l-c------in--yc------o----------------s----------o-v-oz--p-p--l------p
str4: i---g------e---v-a----zg---b--r--------------------------d----d----------bc--s--vr--vn---n-g--------f--
str5: ------p---------y----p--l-r-----------z--x---u-c---p-m-qv-g-t-d-f--u--i-v-cd-sb---o--------------------
str6: ------p--bde---v------------d------

In [ ]:
instance03 = scsp.example.load("uniform_q26n016k015-025.txt")

In [ ]:
bench1(instance03)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 159) ---
  Sol: hiklorstxjkpqwabdfginqxefkouvydghlmnpqsxabjlnrwxzfhipxacovzdgiknpsuybcfjqrvzdkopsvimrtdhloqvxbceoszemtvbcgrvwahknoptxzbdfnpqrtuvxbgiuvcdjsbdhlmoprgwpqxzluwdefy
str01: -------t--k-------g-n----k-u----h-m-p--x----n-----h----------------------------------t----q--------------g----------xz---------vx--i-----s---------------------
str02: -i--o----j---------i-q--f-o------l-n-----b-----x-----x-c-v-------su-----q------p-vi--------------s---------------------------------------sb-

In [ ]:
bench2(instance03)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 147) ---
  Sol: htikrxgpwxybdeknopquvadjklrsxbcfiquvzdfghmopqrxjlnvcfhquyzbcopstmoqvxerwagosxzbcdeirtvdfkpubciknsuvzcdgijkmopqrblpvwxaeinstwzfhkmnsbgoqzcvxbdhmfguy
str01: -t-k--g--------n--------k---------u-----hm-p--x--n---h---------t--q------g--xz-------v------------------------------x--i-s-------------------------
str02: --i-------------o------j--------iq----f---o-----ln--------b---------x-------x--c-----v----------su-----------q---pv----i-s--------sb------x----f---
str03: ---------

オートマトン制約を使うよりも線形制約で定式化した方が良さげ

## ログを見よう

In [ ]:
instance04 = scsp.example.load("uniform_q05n010k010-010.txt")
instance05 = scsp.example.load("uniform_q05n050k010-010.txt")
instance06 = scsp.example.load("nucleotide_n010k010.txt")
instance07 = scsp.example.load("nucleotide_n050k050.txt")
instance08 = scsp.example.load("protein_n010k010.txt")
instance09 = scsp.example.load("protein_n050k050.txt")

In [ ]:
_model = Model(instance01)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x99cb903f0d9752b8)
#Variables: 709 (#bools: 625 in objective) (709 primary variables)
  - 625 Booleans in [0,1]
  - 84 in [0,24]
#kElement: 84
#kLinear2: 80

Starting presolve at 0.00s


  1.34e-04s  0.00e+00d  [DetectDominanceRelations] 
  9.98e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  1.55e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.99e-04s  0.00e+00d  [DetectDuplicateColumns] 
  1.17e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 6'608 nodes and 9'288 arcs.
[Symmetry] Symmetry computation done. time: 0.00102778 dtime: 0.00173038


  6.79e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=662 
  3.11e-02s  9.15e-02d  [Probe] #probed=4'958 #new_binary_clauses=71'762 
  6.91e-06s  0.00e+00d  [MaxClique] 
  6.41e-04s  0.00e+00d  [DetectDominanceRelations] 
  8.27e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=3 #num_dual_strengthening=2 
  1.30e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.04e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  8.23e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.48e-05s  4.80e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=80 
  1.55e-04s  0.00e+00d  [DetectDifferentVariables] #different=45 
  7.23e-05s  3.39e-06d  [ProcessSetPPC] #relevant_constraints=84 
  1.54e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.44e-04s  4.43e-04d  [FindBigAtMostOneAndLinearOverlap] 
  8.36e-05s  4.24e-05d  [FindBigVerticalLinearOverlap] 
  9.92e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.38

  6.33e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.71e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  6.27e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.70e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  5.83e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.05e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 7'060 nodes and 10'547 arcs.
[Symmetry] Symmetry computation done. time: 0.000883115 dtime: 0.00160177
[SAT presolve] num removable Booleans: 0 / 1438
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:969 literals:1938 vars:1276 one_side_vars:1276 simple_definition:0 singleton_clauses:0
[SAT presolve] [3.4466e-05s] clauses:969 literals:1938 vars:1276 one_side_vars:1276 simple_definition:0 singleton_clauses:0
[SAT presolve] [9.0011e-05s] clauses:969 literals:1938 vars:1276 one_side_vars:1276 simple_definition:0 singleton_clauses:0


  9.54e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.20e-02s  9.34e-02d  [Probe] #probed=4'958 #new_binary_clauses=71'646 
  3.57e-03s  3.05e-02d  [MaxClique] Merged 969(1'938 literals) into 663(1'632 literals) at_most_ones. 
  6.14e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.70e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.26e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.04e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  8.45e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.56e-05s  4.80e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=80 
  1.49e-04s  0.00e+00d  [DetectDifferentVariables] #different=45 
  1.88e-04s  8.13e-06d  [ProcessSetPPC] #relevant_constraints=747 
  2.46e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.24e-04s  3.23e-04d  [FindBigAtMostOneAndLinearOverlap] 
  8.50e-05s  4.02e-05d  [FindBigVerticalLinearOverlap] 
  1.24e-05s  0.00e+00d 

...

#Bound   6.13s best:63    next:[61,62]    bool_core (num_cores=45 [size:1 mw:1] a=7 d=13 fixed=2/3334 clauses=25'958)


#Bound  12.59s best:63    next:[62,62]    bool_core (num_cores=46 [size:1 mw:1] a=1 d=13 fixed=3/3382 clauses=31'134)


#Model  12.63s var:1415/1449 constraints:2781/2850


#12     15.21s best:62    next:[]         core
#Done   15.21s core



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [  15.04s,   15.04s]   15.04s   0.00ns   15.04s         1 [  20.85s,   20.85s]   20.85s   0.00ns   20.85s
           'default_lp':         1 [  15.04s,   15.04s]   15.04s   0.00ns   15.04s         1 [   6.03s,    6.03s]    6.03s   0.00ns    6.03s
     'feasibility_pump':        64 [  6.43ms,  13.25ms]  11.80ms 813.43us 755.32ms        63 [  2.97ms,   4.37ms]   3.04ms 170.64us 191.46ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [  1.87ms,   1.87ms]   1.87ms   0.00ns   1.87ms         1 [260.70us, 260.70us] 260.70us   0.00ns 260.70us
            'fs_random':         1 [  4.90ms,   4.90ms]   4.90ms   0.00ns   4.90ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance02)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x62caf4921ff464ba)
#Variables: 825 (#bools: 650 in objective) (825 primary variables)
  - 650 Booleans in [0,1]
  - 175 in [0,24]
#kElement: 175
#kLinear2: 167

Starting presolve at 0.01s


  1.84e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.13e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  1.83e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  4.24e-04s  0.00e+00d  [DetectDuplicateColumns] 
  3.85e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 13'564 nodes and 20'069 arcs.
[Symmetry] Symmetry computation done. time: 0.00230293 dtime: 0.00411182


  1.79e-03s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=1'877 
  7.24e-02s  2.22e-01d  [Probe] #probed=9'872 #new_binary_clauses=169'657 
  1.56e-05s  0.00e+00d  [MaxClique] 


  1.31e-03s  0.00e+00d  [DetectDominanceRelations] 
  1.68e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=3 #num_dual_strengthening=2 
  2.78e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  3.74e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.53e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.65e-05s  1.00e-06d  [DetectDominatedLinearConstraints] #relevant_constraints=167 
  2.64e-04s  0.00e+00d  [DetectDifferentVariables] #different=87 
  1.48e-04s  7.34e-06d  [ProcessSetPPC] #relevant_constraints=175 
  3.18e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  7.35e-04s  9.84e-04d  [FindBigAtMostOneAndLinearOverlap] 
  1.77e-04s  9.23e-05d  [FindBigVerticalLinearOverlap] 
  2.27e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  2.81e-05s  0.00e+00d  [MergeClauses] 


  1.30e-03s  0.00e+00d  [DetectDominanceRelations] 
  8.03e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.29e-03s  0.00e+00d  [DetectDominanceRelations] 
  8.02e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.23e-04s  0.00e+00d  [DetectDuplicateColumns] 
  3.81e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 14'375 nodes and 23'108 arcs.


...

#Bound  20.83s best:102   next:[73,101]   bool_core (num_cores=70 [size:11 mw:1 amo:1 lit:2 d:15] a=16 d=15 fixed=1/6732 clauses=36'906)


#Bound  29.88s best:102   next:[74,101]   bool_core (num_cores=71 [size:5 mw:1 amo:1 lit:2 d:16] a=12 d=16 fixed=1/6837 clauses=47'790)


#Bound  46.41s best:102   next:[75,101]   bool_core (num_cores=72 [size:7 mw:1 amo:1 lit:2 d:17] a=6 d=17 fixed=1/6934 clauses=55'088)


#Bound  79.21s best:102   next:[76,101]   bool_core (num_cores=73 [size:3 mw:1 d:18] a=4 d=18 fixed=1/7017 clauses=66'429)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.99m,    1.99m]    1.99m   0.00ns    1.99m         1 [   1.93m,    1.93m]    1.93m   0.00ns    1.93m
           'default_lp':         1 [   1.99m,    1.99m]    1.99m   0.00ns    1.99m         1 [  37.18s,   37.18s]   37.18s   0.00ns   37.18s
     'feasibility_pump':       529 [ 14.58us,  14.65ms] 233.28us 643.69us 123.40ms       524 [ 41.91us,   2.48ms]  46.57us 106.55us  24.40ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [  4.59ms,   4.59ms]   4.59ms   0.00ns   4.59ms         1 [591.54us, 591.54us] 591.54us   0.00ns 591.54us
            'fs_random':         1 [  5.29ms,   5.29ms]   5.29ms   0.00ns   5.29ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance03)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12



Initial optimization model '': (model_fingerprint: 0x6fcf0a5bda8383ae)
#Variables: 973 (#bools: 650 in objective) (973 primary variables)
  - 650 Booleans in [0,1]
  - 323 in [0,24]
#kElement: 323
#kLinear2: 307

Starting presolve at 0.01s


  2.61e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.69e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  2.89e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  1.05e-03s  0.00e+00d  [DetectDuplicateColumns] 
  6.67e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 26'105 nodes and 39'633 arcs.
[Symmetry] Symmetry computation done. time: 0.00483572 dtime: 0.00911198


  3.99e-03s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=4'212 
  1.42e-01s  4.38e-01d  [Probe] #probed=18'672 #new_binary_clauses=332'985 
  5.39e-05s  0.00e+00d  [MaxClique] 


...

#Bound  35.93s best:138   next:[81,137]   bool_core (num_cores=81 [size:12 mw:1 amo:1 lit:3 d:14] a=9 d=14 fixed=3/11572 clauses=49'050)


#Bound  47.40s best:138   next:[82,137]   bool_core (num_cores=82 [size:4 mw:1 amo:1 lit:2 d:15] a=6 d=15 fixed=3/11705 clauses=46'820)


#54     49.87s best:137   next:[82,136]   rnd_cst_lns (d=7.78e-01 s=1360 t=0.10 p=0.52 stall=78 h=base)


#Bound  78.76s best:137   next:[83,136]   bool_core (num_cores=83 [size:5 mw:1 amo:1 lit:2 d:16] a=2 d=16 fixed=3/11823 clauses=56'475)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.99m,    1.99m]    1.99m   0.00ns    1.99m         1 [   1.43m,    1.43m]    1.43m   0.00ns    1.43m
           'default_lp':         1 [   1.99m,    1.99m]    1.99m   0.00ns    1.99m         1 [  49.31s,   49.31s]   49.31s   0.00ns   49.31s
     'feasibility_pump':       525 [ 55.15us,  28.67ms] 350.09us   1.26ms 183.80ms       522 [ 60.92us,   3.33ms]  67.19us 143.15us  35.07ms
                   'fj':         1 [ 10.78ms,  10.78ms]  10.78ms   0.00ns  10.78ms         1 [  1.15ms,   1.15ms]   1.15ms   0.00ns   1.15ms
                   'fj':         1 [ 26.85ms,  26.85ms]  26.85ms   0.00ns  26.85ms         1 [ 12.79ms,  12.79ms]  12.79ms   0.00ns  12.79ms
            'fs_random':         1 [ 47.33ms,  47.33ms]  47.33ms   0.00ns  47.33ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance04)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xbeccfc87c55e4ee)
#Variables: 150 (#bools: 50 in objective) (150 primary variables)
  - 50 Booleans in [0,1]
  - 100 in [0,9]
#kElement: 100
#kLinear2: 90

Starting presolve at 0.00s
  4.17e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.51e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  2.01e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  8.92e-05s  0.00e+00d  [DetectDuplicateColumns] 
  5.62e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'780 nodes and 4'030 arcs.
[Symmetry] Symmetry computation done. time: 0.000504607 dtime: 0.0009608
  1.81e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=420 
  4.19e-03s  4.32e-03d  [Probe] #probed=1'380 #new_binary_clauses=4'590 
  5.65e-06s  0.00e+00d  [M

  2.24e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.00e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  6.87e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  3.68e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  2.56e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.75e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  1.04e-04s  0.00e+00d  [DetectDifferentVariables] #different=53 
  5.29e-05s  1.41e-06d  [ProcessSetPPC] #relevant_constraints=100 
  7.68e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  7.55e-05s  7.92e-05d  [FindBigAtMostOneAndLinearOverlap] 
  2.36e-05s  1.94e-05d  [FindBigVerticalLinearOverlap] 
  4.00e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  7.11e-06s  0.00e+00d  [MergeClauses] 
  2.20e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.46e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.58e-04s  0.00e+00d  [DetectDominanceRela

  3.71e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.05e-03s  4.42e-03d  [Probe] #probed=1'380 #new_binary_clauses=4'783 
  4.23e-04s  1.07e-03d  [MaxClique] 
  2.18e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.46e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  6.91e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  4.23e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  3.71e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.30e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  9.31e-05s  0.00e+00d  [DetectDifferentVariables] #different=53 
  1.35e-04s  3.72e-06d  [ProcessSetPPC] #relevant_constraints=391 
  1.28e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  8.52e-05s  7.23e-05d  [FindBigAtMostOneAndLinearOverlap] 
  3.10e-05s  1.85e-05d  [FindBigVerticalLinearOverlap] 
  6.29e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.50e-05s  0.00e+00d  [MergeCl

#2       0.05s best:34    next:[0,33]     no_lp
#Bound   0.05s best:34    next:[1,33]     bool_core (num_cores=1 [size:4 mw:1 d:2] a=47 d=2 fixed=0/692 clauses=372)
#3       0.05s best:32    next:[1,31]     quick_restart
#4       0.06s best:31    next:[1,30]     no_lp


#Bound   0.06s best:31    next:[2,30]     bool_core (num_cores=2 [size:4 mw:1 d:2] a=44 d=2 fixed=0/697 clauses=314)
#Bound   0.06s best:31    next:[3,30]     bool_core (num_cores=3 [size:3 mw:1 d:2] a=42 d=2 fixed=0/701 clauses=318)
#Bound   0.06s best:31    next:[4,30]     bool_core (num_cores=4 [size:5 mw:1 d:3] a=38 d=3 fixed=0/706 clauses=323)
#Bound   0.06s best:31    next:[5,30]     bool_core (num_cores=5 [size:4 mw:1 d:2] a=35 d=3 fixed=0/712 clauses=329)
#Bound   0.06s best:31    next:[6,30]     bool_core (num_cores=6 [size:3 mw:1 d:2] a=33 d=3 fixed=0/716 clauses=336)
#Bound   0.06s best:31    next:[7,30]     bool_core (num_cores=7 [size:4 mw:1 d:3] a=30 d=3 fixed=0/720 clauses=341)
#Bound   0.06s best:31    next:[8,30]     bool_core (num_cores=8 [size:3 mw:1 d:2] a=28 d=3 fixed=0/725 clauses=349)
#Bound   0.07s best:31    next:[9,30]     bool_core (num_cores=9 [size:2 mw:1 d:3] a=27 d=3 fixed=0/727 clauses=354)
#Bound   0.07s best:31    next:[10,30]    reduced_costs
#Bound  

#5       0.07s best:30    next:[27,29]    no_lp


#Bound   0.10s best:30    next:[28,29]    bool_core (num_cores=24 [size:2 mw:1 d:8] a=2 d=8 fixed=4/816 clauses=1'020)


#Bound   0.14s best:30    next:[29,29]    bool_core (num_cores=25 [size:2 mw:1 exo] a=0 d=8 fixed=4/831 clauses=1'015)
#6       0.14s best:29    next:[]         core
#Done    0.14s core

Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [106.06ms, 106.06ms] 106.06ms   0.00ns 106.06ms         1 [ 77.18ms,  77.18ms]  77.18ms   0.00ns  77.18ms
           'default_lp':         1 [107.01ms, 107.01ms] 107.01ms   0.00ns 107.01ms         1 [ 62.34ms,  62.34ms]  62.34ms   0.00ns  62.34ms
     'feasibility_pump':         2 [462.50us,   2.50ms]   1.48ms   1.02ms   2.97ms         1 [ 70.65us,  70.65us]  70.65us   0.00ns  70.65us
                   'fj':         1 [ 18.80ms,  18.80ms]  18.80ms   0.00ns  18.80ms         1 [ 12.99ms,  12.99ms]  12.99ms   0.00ns  12.99ms
                   'fj':         1 [819.90us, 819.90us] 819.90us   0.00ns 819.90us         1 [165.88us, 165.

In [ ]:
_model = Model(instance05)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xb402e8729ca35680)
#Variables: 550 (#bools: 50 in objective) (550 primary variables)
  - 50 Booleans in [0,1]
  - 500 in [0,9]
#kElement: 500
#kLinear2: 450

Starting presolve at 0.00s


  1.52e-04s  0.00e+00d  [DetectDominanceRelations] 
  7.67e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  3.28e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  4.29e-04s  0.00e+00d  [DetectDuplicateColumns] 
  1.72e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 13'550 nodes and 19'910 arcs.


[Symmetry] Symmetry computation done. time: 0.00329766 dtime: 0.00620883


  8.32e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=2'270 
  1.91e-02s  2.08e-02d  [Probe] #probed=6'380 #new_binary_clauses=22'230 
  1.65e-05s  0.00e+00d  [MaxClique] 


  1.04e-03s  0.00e+00d  [DetectDominanceRelations] 
  1.44e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.72e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.59e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.12e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  7.41e-05s  2.70e-06d  [DetectDominatedLinearConstraints] #relevant_constraints=450 
  3.48e-04s  0.00e+00d  [DetectDifferentVariables] #different=268 
  1.74e-04s  6.96e-06d  [ProcessSetPPC] #relevant_constraints=500 
  3.26e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.59e-04s  3.85e-04d  [FindBigAtMostOneAndLinearOverlap] 
  1.27e-04s  9.50e-05d  [FindBigVerticalLinearOverlap] 
  2.34e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  3.21e-05s  0.00e+00d  [MergeClauses] 
  1.03e-03s  0.00e+00d  [DetectDominanceRelations] 
  7.17e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 


...

#Bound   0.38s best:36    next:[18,35]    bool_core (num_cores=18 [size:3 mw:1 d:5] a=13 d=5 fixed=0/3248 clauses=1'627)
#Bound   0.39s best:36    next:[19,35]    bool_core (num_cores=19 [size:3 mw:1 d:4] a=11 d=5 fixed=0/3253 clauses=1'638)
#Bound   0.39s best:36    next:[20,35]    bool_core (num_cores=20 [size:2 mw:1 d:5] a=10 d=5 fixed=0/3257 clauses=1'651)


#Bound   0.40s best:36    next:[21,35]    bool_core (num_cores=21 [size:2 mw:1 d:6] a=9 d=6 fixed=0/3261 clauses=1'673)
#Bound   0.40s best:36    next:[22,35]    bool_core (num_cores=22 [size:2 mw:1 d:3] a=8 d=6 fixed=0/3266 clauses=1'688)


#4       0.55s best:35    next:[33,34]    quick_restart


#5       0.90s best:34    next:[]         core
#Done    0.90s core


#Bound   0.84s best:35    next:[34,34]    bool_core (num_cores=30 [size:1 mw:1] a=1 d=10 fixed=4/3318 clauses=2'807) [skipped_logs=11]

Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [721.32ms, 721.32ms] 721.32ms   0.00ns 721.32ms         1 [441.33ms, 441.33ms] 441.33ms   0.00ns 441.33ms
           'default_lp':         1 [723.88ms, 723.88ms] 723.88ms   0.00ns 723.88ms         1 [218.04ms, 218.04ms] 218.04ms   0.00ns 218.04ms
     'feasibility_pump':         5 [277.97us,  12.85ms]   3.26ms   4.88ms  16.28ms         4 [ 66.08us, 638.71us] 209.24us 247.96us 836.96us
                   'fj':         1 [  5.28ms,   5.28ms]   5.28ms   0.00ns   5.28ms         1 [746.85us, 746.85us] 746.85us   0.00ns 746.85us
                   'fj':         1 [ 18.98ms,  18.98ms]  18.98ms   0.00ns  18.98ms         1 [  4.45ms,   4.45ms]   4.45ms   0.00ns   4.45ms
            'fs_ra

In [ ]:
_model = Model(instance06)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x637912b6fd74a00e)
#Variables: 140 (#bools: 40 in objective) (140 primary variables)
  - 40 Booleans in [0,1]
  - 100 in [0,9]
#kElement: 100
#kLinear2: 90

Starting presolve at 0.00s
  4.29e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.44e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  1.44e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  7.32e-05s  0.00e+00d  [DetectDuplicateColumns] 
  4.43e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'470 nodes and 3'550 arcs.
[Symmetry] Symmetry computation done. time: 0.000461125 dtime: 0.0008805
  1.44e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=370 
  3.37e-03s  3.01e-03d  [Probe] #probed=1'120 #new_binary_clauses=3'375 
  4.01e-06s  0.00e+00d  [

  6.82e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  3.60e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  2.29e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.74e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  9.40e-05s  0.00e+00d  [DetectDifferentVariables] #different=59 
  4.58e-05s  1.23e-06d  [ProcessSetPPC] #relevant_constraints=100 
  6.35e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  6.08e-05s  6.14e-05d  [FindBigAtMostOneAndLinearOverlap] 
  2.03e-05s  1.72e-05d  [FindBigVerticalLinearOverlap] 
  3.61e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.03e-05s  0.00e+00d  [MergeClauses] 
  1.95e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.29e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.98e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.29e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  3.98e-05s  0.00e+00d  [DetectDuplicateColu

  1.92e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.27e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.87e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.26e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  4.54e-05s  0.00e+00d  [DetectDuplicateColumns] 
  2.85e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'382 nodes and 3'591 arcs.
[Symmetry] Symmetry computation done. time: 0.000332551 dtime: 0.0007131
[SAT presolve] num removable Booleans: 0 / 448
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:131 literals:262 vars:168 one_side_vars:168 simple_definition:0 singleton_clauses:0
[SAT presolve] [9.719e-06s] clauses:131 literals:262 vars:168 one_side_vars:168 simple_definition:0 singleton_clauses:0
[SAT presolve] [2.8935e-05s] clauses:131 literals:262 vars:168 one_side_vars:168 simple_definition:0 singleton_clauses:0
  3.02e-05s  0.00e+00d  [DetectDuplicateConstrain

#Model   0.03s var:548/548 constraints:1163/1163

Starting search at 0.03s with 12 workers.
8 full problem subsolvers: [core, default_lp, max_lp, no_lp, pseudo_costs, quick_restart, quick_restart_no_lp, reduced_costs]
4 first solution subsolvers: [fj(2), fs_random, fs_random_no_lp]
10 interleaved subsolvers: [feasibility_pump, graph_arc_lns, graph_cst_lns, graph_dec_lns, graph_var_lns, ls, ls_lin, rins/rens, rnd_cst_lns, rnd_var_lns]
3 helper subsolvers: [neighborhood_helper, synchronization_agent, update_gap_integral]

#1       0.04s best:29    next:[0,28]     fj_restart(batch:1 lin{mvs:127 evals:476} #w_updates:0 #perturb:0)
#2       0.04s best:28    next:[0,27]     no_lp
#Bound   0.04s best:28    next:[1,27]     bool_core (num_cores=1 [size:4 mw:1 d:2] a=37 d=2 fixed=0/560 clauses=312)


#3       0.04s best:27    next:[1,26]     default_lp
#4       0.05s best:26    next:[1,25]     no_lp
#Bound   0.05s best:26    next:[2,25]     bool_core (num_cores=2 [size:4 mw:1 d:2] a=34 d=2 fixed=0/565 clauses=252)
#Bound   0.05s best:26    next:[3,25]     bool_core (num_cores=3 [size:3 mw:1 d:2] a=32 d=2 fixed=0/569 clauses=256)
#Bound   0.05s best:26    next:[4,25]     bool_core (num_cores=4 [size:3 mw:1 d:2] a=30 d=2 fixed=0/572 clauses=259)
#Bound   0.05s best:26    next:[5,25]     bool_core (num_cores=5 [size:3 mw:1 d:3] a=28 d=3 fixed=0/575 clauses=262)
#Bound   0.05s best:26    next:[6,25]     bool_core (num_cores=6 [size:3 mw:1 d:2] a=26 d=3 fixed=0/579 clauses=267)
#5       0.05s best:25    next:[6,24]     no_lp
#Bound   0.05s best:25    next:[7,24]     bool_core (num_cores=7 [size:4 mw:1 d:2] a=23 d=3 fixed=0/583 clauses=271)
#Bound   0.05s best:25    next:[8,24]     bool_core (num_cores=8 [size:3 mw:1 d:2] a=21 d=3 fixed=0/587 clauses=275)
#Bound   0.05s best:25    next:[

#Bound   0.06s best:25    next:[12,24]    bool_core (num_cores=12 [size:3 mw:1 d:3] a=12 d=3 fixed=0/601 clauses=293)
#Bound   0.06s best:25    next:[13,24]    bool_core (num_cores=13 [size:2 mw:1 d:4] a=11 d=4 fixed=0/604 clauses=303)
#Bound   0.06s best:25    next:[14,24]    bool_core (num_cores=14 [size:3 mw:1 d:3] a=9 d=4 fixed=0/609 clauses=313)
#Bound   0.06s best:25    next:[15,24]    bool_core (num_cores=15 [size:2 mw:1 d:3] a=8 d=4 fixed=0/612 clauses=319)
#Bound   0.06s best:25    next:[16,24]    bool_core (num_cores=16 [size:2 mw:1 d:4] a=7 d=4 fixed=0/615 clauses=323)
#Bound   0.06s best:25    next:[17,24]    bool_core (num_cores=17 [size:2 mw:1 d:5] a=6 d=5 fixed=0/618 clauses=339)
#Bound   0.06s best:25    next:[18,24]    bool_core (num_cores=18 [size:2 mw:1 d:5] a=5 d=5 fixed=0/624 clauses=383)
#Bound   0.06s best:25    next:[19,24]    bool_core (num_cores=19 [size:2 mw:1 d:6] a=4 d=6 fixed=0/628 clauses=434)
#Bound   0.06s best:25    next:[20,24]    bool_core (num_cores

#6       0.15s best:24    next:[]         core
#Done    0.15s core
#Bound   0.15s best:25    next:[24,24]    bool_core (num_cores=23 [size:1 mw:1] a=1 d=8 fixed=1/659 clauses=944) [skipped_logs=3]

Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [119.50ms, 119.50ms] 119.50ms   0.00ns 119.50ms         1 [105.77ms, 105.77ms] 105.77ms   0.00ns 105.77ms
           'default_lp':         1 [120.25ms, 120.25ms] 120.25ms   0.00ns 120.25ms         1 [ 80.04ms,  80.04ms]  80.04ms   0.00ns  80.04ms
     'feasibility_pump':         4 [ 69.52us,   2.30ms] 757.14us 914.58us   3.03ms         3 [ 14.33us, 169.34us]  66.00us  73.07us 197.99us
                   'fj':         1 [ 10.40ms,  10.40ms]  10.40ms   0.00ns  10.40ms         1 [  6.52ms,   6.52ms]   6.52ms   0.00ns   6.52ms
                   'fj':         1 [806.76us, 806.76us] 806.76us   0.00ns 806.76us         1 [149

In [ ]:
_model = Model(instance07)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0x9a73ef9c5835379)
#Variables: 2'750 (#bools: 250 in objective) (2'750 primary variables)
  - 250 Booleans in [0,1]
  - 2'500 in [0,49]
#kElement: 2'500
#kLinear2: 2'450

Starting presolve at 0.04s


  1.50e-03s  0.00e+00d  [DetectDominanceRelations] 
  4.31e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=50 #num_dual_strengthening=1 
  3.00e-05s  0.00e+00d  [ExtractEncodingFromLinear] 


  9.78e-03s  0.00e+00d  [DetectDuplicateColumns] 
  5.16e-03s  0.00e+00d  [DetectDuplicateConstraints] 


[Symmetry] Graph for symmetry has 242'150 nodes and 378'550 arcs.


...

#Bound  96.51s best:136   next:[49,135]   bool_core (num_cores=31 [cover] a=8 d=9 fixed=17/86182 clauses=56'018)


#Bound 105.13s best:136   next:[50,135]   bool_core (num_cores=32 [size:1 mw:1] a=8 d=9 fixed=18/86191 clauses=58'211)


#Bound 111.08s best:136   next:[51,135]   bool_core (num_cores=33 [size:2 mw:1 d:7] a=7 d=9 fixed=19/86197 clauses=54'817)


#Bound 112.63s best:136   next:[52,135]   bool_core (num_cores=34 [size:1 mw:1] a=7 d=9 fixed=20/86203 clauses=55'192)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.92m,    1.92m]    1.92m   0.00ns    1.92m         1 [  35.35s,   35.35s]   35.35s   0.00ns   35.35s
           'default_lp':         1 [   1.92m,    1.92m]    1.92m   0.00ns    1.92m         1 [  25.22s,   25.22s]   25.22s   0.00ns   25.22s
     'feasibility_pump':       484 [ 79.04us, 365.65ms]   2.66ms  16.67ms    1.29s       476 [341.99us,   5.70ms] 356.95us 258.19us 169.91ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [206.00ms, 206.00ms] 206.00ms   0.00ns 206.00ms         1 [ 11.22ms,  11.22ms]  11.22ms   0.00ns  11.22ms
            'fs_random':         1 [251.26ms, 251.26ms] 251.26ms   0.00ns 251.26ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance08)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x83e688bb8355a5e0)
#Variables: 290 (#bools: 190 in objective) (290 primary variables)
  - 190 Booleans in [0,1]
  - 100 in [0,9]
#kElement: 100
#kLinear2: 90

Starting presolve at 0.00s
  6.23e-05s  0.00e+00d  [DetectDominanceRelations] 
  2.89e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  1.43e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  9.71e-05s  0.00e+00d  [DetectDuplicateColumns] 
  5.09e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 3'120 nodes and 4'350 arcs.
[Symmetry] Symmetry computation done. time: 0.00057964 dtime: 0.00104613


  1.70e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] #without_enforcements=354 
  4.87e-03s  5.23e-03d  [Probe] #probed=1'752 #new_binary_clauses=5'400 
  4.22e-06s  0.00e+00d  [MaxClique] 
  2.78e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.56e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=3 #num_dual_strengthening=2 
  7.03e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  4.80e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  3.53e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.77e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  9.73e-05s  0.00e+00d  [DetectDifferentVariables] #different=49 
  4.62e-05s  1.53e-06d  [ProcessSetPPC] #relevant_constraints=100 
  7.20e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  9.98e-05s  9.10e-05d  [FindBigAtMostOneAndLinearOverlap] 
  3.67e-05s  2.09e-05d  [FindBigVerticalLinearOverlap] 
  4.27e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  8.12

  4.93e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.65e-03s  5.11e-03d  [Probe] #probed=1'752 #new_binary_clauses=5'105 
  4.57e-04s  1.18e-03d  [MaxClique] Merged 472(944 literals) into 421(893 literals) at_most_ones. 
  2.87e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.68e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  6.46e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  5.20e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  4.07e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.75e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  9.37e-05s  0.00e+00d  [DetectDifferentVariables] #different=49 
  1.11e-04s  4.24e-06d  [ProcessSetPPC] #relevant_constraints=521 
  1.42e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  1.21e-04s  8.07e-05d  [FindBigAtMostOneAndLinearOverlap] 
  3.75e-05s  2.07e-05d  [FindBigVerticalLinearOverlap] 
  5.02e-06s  0.00e+00d  [Fin

  4.56e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.71e-03s  5.13e-03d  [Probe] #probed=1'752 #new_binary_clauses=5'126 
  4.52e-04s  1.19e-03d  [MaxClique] 
  2.82e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.67e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  7.19e-05s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  5.34e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  4.20e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.92e-05s  5.40e-07d  [DetectDominatedLinearConstraints] #relevant_constraints=90 
  9.38e-05s  0.00e+00d  [DetectDifferentVariables] #different=49 
  1.11e-04s  4.24e-06d  [ProcessSetPPC] #relevant_constraints=521 
  1.56e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  1.15e-04s  8.15e-05d  [FindBigAtMostOneAndLinearOverlap] 
  3.73e-05s  2.07e-05d  [FindBigVerticalLinearOverlap] 
  7.13e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.46e-05s  0.00e+00d  [MergeCl

#1       0.05s best:61    next:[1,60]     fj_restart(batch:1 lin{mvs:149 evals:595} #w_updates:0 #perturb:0)
#2       0.06s best:60    next:[1,59]     no_lp
#Bound   0.06s best:60    next:[2,59]     bool_core (num_cores=1 [size:5 mw:1 amo:1 lit:2 d:3] a=147 d=3 fixed=0/840 clauses=413)
#3       0.06s best:59    next:[2,58]     graph_cst_lns (d=5.00e-01 s=17 t=0.10 p=0.00 stall=0 h=base) [hint] [combined with: graph_arc_lns (d=5.0...]
#4       0.06s best:58    next:[2,57]     rnd_var_lns (d=7.07e-01 s=21 t=0.10 p=1.00 stall=1 h=base)


...

#Bound   0.48s best:48    next:[44,47]    bool_core (num_cores=42 [size:2 mw:1 d:11] a=16 d=11 fixed=1/1191 clauses=6'229)


#Bound   0.86s best:48    next:[45,47]    bool_core (num_cores=43 [size:3 mw:1 d:12] a=14 d=12 fixed=1/1213 clauses=13'987)


#Bound   1.86s best:48    next:[46,47]    bool_core (num_cores=44 [size:6 mw:1 d:13] a=9 d=13 fixed=1/1242 clauses=15'726)


#14      1.92s best:46    next:[]         core
#Done    1.92s core



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.87s,    1.87s]    1.87s   0.00ns    1.87s         1 [   3.06s,    3.06s]    3.06s   0.00ns    3.06s
           'default_lp':         1 [   1.87s,    1.87s]    1.87s   0.00ns    1.87s         1 [877.58ms, 877.58ms] 877.58ms   0.00ns 877.58ms
     'feasibility_pump':        12 [ 98.79us,   2.86ms] 419.18us 783.31us   5.03ms        11 [ 20.56us, 517.41us]  65.72us 142.84us 722.97us
                   'fj':         1 [  1.12ms,   1.12ms]   1.12ms   0.00ns   1.12ms         1 [174.17us, 174.17us] 174.17us   0.00ns 174.17us
                   'fj':         1 [  2.50ms,   2.50ms]   2.50ms   0.00ns   2.50ms         1 [  1.22ms,   1.22ms]   1.22ms   0.00ns   1.22ms
            'fs_random':         1 [  5.57ms,   5.57ms]   5.57ms   0.00ns   5.57ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance09)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0x47233af4258049d6)
#Variables: 3'500 (#bools: 1'000 in objective) (3'500 primary variables)
  - 1'000 Booleans in [0,1]
  - 2'500 in [0,49]
#kElement: 2'500
#kLinear2: 2'450

Starting presolve at 0.14s


  1.99e-03s  0.00e+00d  [DetectDominanceRelations] 
  9.48e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=50 #num_dual_strengthening=1 
  4.36e-05s  0.00e+00d  [ExtractEncodingFromLinear] 


  1.07e-02s  0.00e+00d  [DetectDuplicateColumns] 
  7.29e-03s  0.00e+00d  [DetectDuplicateConstraints] 


[Symmetry] Graph for symmetry has 303'150 nodes and 474'950 arcs.


...

#Bound 117.00s best:437   next:[68,436]   bool_core (num_cores=53 [size:4 mw:1 d:6] a=160 d=7 fixed=15/112349 clauses=63'256)


#Bound 117.48s best:437   next:[69,436]   bool_core (num_cores=54 [size:2 mw:1 d:6] a=159 d=7 fixed=15/112361 clauses=63'307)


#Bound 118.42s best:437   next:[70,436]   bool_core (num_cores=55 [size:4 mw:1 d:6] a=156 d=7 fixed=15/112373 clauses=63'392)


#Bound 119.28s best:437   next:[71,436]   bool_core (num_cores=56 [size:2 mw:1 d:6] a=155 d=7 fixed=15/112386 clauses=63'482)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.89m,    1.89m]    1.89m   0.00ns    1.89m         1 [  30.39s,   30.39s]   30.39s   0.00ns   30.39s
           'default_lp':         1 [   1.90m,    1.90m]    1.90m   0.00ns    1.90m         1 [  20.93s,   20.93s]   20.93s   0.00ns   20.93s
     'feasibility_pump':       486 [ 80.54us, 358.66ms]   2.25ms  16.22ms    1.09s       473 [354.60us,   5.75ms] 377.17us 346.32us 178.40ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [223.02ms, 223.02ms] 223.02ms   0.00ns 223.02ms         1 [ 13.42ms,  13.42ms]  13.42ms   0.00ns  13.42ms
            'fs_random':         1 [228.13ms, 228.13ms] 228.13ms   0.00ns 228.13ms         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

巨大なインスタンスで試してみよう

In [ ]:
instance_large1 = scsp.example.load("nucleotide_n100k100.txt")
instance_large2 = scsp.example.load("protein_n100k100.txt")

In [ ]:
_model = Model(instance_large1)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0xbacd301d4dee94a1)
#Variables: 10'600 (#bools: 600 in objective) (10'600 primary variables)
  - 600 Booleans in [0,1]
  - 10'000 in [0,99]
#kElement: 10'000
#kLinear2: 9'900

Starting presolve at 0.34s


  9.51e-03s  0.00e+00d  [DetectDominanceRelations] 
  6.49e+00s  0.00e+00d  [PresolveToFixPoint] #num_loops=100 #num_dual_strengthening=1 
  7.51e-05s  0.00e+00d  [ExtractEncodingFromLinear] 


  6.90e-02s  0.00e+00d  [DetectDuplicateColumns] 


  5.34e-02s  0.00e+00d  [DetectDuplicateConstraints] 


...

#Bound  79.26s best:281   next:[4,280]    bool_core (num_cores=2 [size:26 mw:1 d:5] a=350 d=5 fixed=0/717073 clauses=363'373)


#7      96.18s best:279   next:[4,278]    pseudo_costs


#Bound  97.51s best:279   next:[5,278]    bool_core (num_cores=3 [size:26 mw:1 d:5] a=325 d=5 fixed=0/717122 clauses=363'422)


#Bound 118.03s best:279   next:[6,278]    bool_core (num_cores=4 [size:26 mw:1 d:5] a=300 d=5 fixed=0/717171 clauses=363'471)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.45m,    1.45m]    1.45m   0.00ns    1.45m         1 [  16.66s,   16.66s]   16.66s   0.00ns   16.66s
           'default_lp':         1 [   1.45m,    1.45m]    1.45m   0.00ns    1.45m         1 [  10.39s,   10.39s]   10.39s   0.00ns   10.39s
     'feasibility_pump':       363 [307.99us,    2.93s]  14.46ms 153.23ms    5.25s       337 [  1.37ms,  14.10ms]   1.54ms   1.34ms 518.09ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         1 [   1.53s,    1.53s]    1.53s   0.00ns    1.53s         1 [ 85.62ms,  85.62ms]  85.62ms   0.00ns  85.62ms
            'fs_random':         1 [   1.56s,    1.56s]    1.56s   0.00ns    1.56s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance_large2)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0x9d2ad25dcaf63b62)
#Variables: 12'000 (#bools: 2'000 in objective) (12'000 primary variables)
  - 2'000 Booleans in [0,1]
  - 10'000 in [0,99]
#kElement: 10'000
#kLinear2: 9'900

Starting presolve at 1.12s


  1.15e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.27e+01s  0.00e+00d  [PresolveToFixPoint] #num_loops=100 #num_dual_strengthening=1 
  7.61e-05s  0.00e+00d  [ExtractEncodingFromLinear] 


  8.91e-02s  0.00e+00d  [DetectDuplicateColumns] 


  6.97e-02s  0.00e+00d  [DetectDuplicateConstraints] 


...

#15     71.98s best:1024  next:[0,1023]   no_lp


#16     72.40s best:1023  next:[0,1022]   ls_restart_perturb(batch:1 lin{mvs:142 evals:3'610} #w_updates:158 #perturb:0)


#Bound  76.00s best:1023  next:[1,1022]   bool_core (num_cores=1 [size:45 mw:1 d:6] a=1944 d=6 fixed=0/926229 clauses=467'143)


#Bound 105.44s best:1023  next:[2,1022]   bool_core (num_cores=2 [size:41 mw:1 d:6] a=1904 d=6 fixed=0/926312 clauses=467'227)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.23m,    1.23m]    1.23m   0.00ns    1.23m         1 [  17.75s,   17.75s]   17.75s   0.00ns   17.75s
           'default_lp':         1 [   1.19m,    1.19m]    1.19m   0.00ns    1.19m         1 [   8.92s,    8.92s]    8.92s   0.00ns    8.92s
     'feasibility_pump':       283 [300.57us,    3.73s]  21.48ms 221.06ms    6.08s       246 [  1.38ms,  19.17ms]   1.62ms   1.75ms 399.16ms
                   'fj':         1 [   1.66s,    1.66s]    1.66s   0.00ns    1.66s         1 [102.47ms, 102.47ms] 102.47ms   0.00ns 102.47ms
                   'fj':         2 [823.72ms,    1.49s]    1.16s 332.89ms    2.31s         2 [102.46ms, 124.16ms] 113.31ms  10.85ms 226.62ms
            'fs_random':         1 [   6.16s,    6.16s]    6.16s   0.00ns    6.16s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

大きなインスタンスに対しても実行可能解がきちんと出る.
Dual Bound はカスだけど...